<a href="https://colab.research.google.com/github/mmouree/Data-mining-project/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

#######################################
### PRE-TRAINED MODELS AVAILABLE HERE
## https://pytorch.org/docs/stable/torchvision/models.html
from torchvision import models
#######################################

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [3]:
##########################
### SETTINGS
##########################

# Device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)

NUM_CLASSES = 10

# Hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 10
batch_size = 128


##########################
### MNIST DATASET
##########################

custom_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

## Note that this particular normalization scheme is
## necessary since it was used for pre-training
## the network on ImageNet.
## These are the channel-means and standard deviations
## for z-score normalization.


train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=custom_transform,
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=custom_transform)


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          num_workers=2,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         num_workers=2,
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Device: cuda:0


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Image batch dimensions: torch.Size([128, 3, 224, 224])
Image label dimensions: torch.Size([128])


In [4]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


"Creating network Module"
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(3,64 , 5)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.conv3 = nn.Conv2d(128, 256, 5)
        
        self.fc1 = nn.Linear(in_features = 256*24*24, out_features = 512)
        self.fc2 = nn.Linear(in_features = 512, out_features = 1024)
        self.out = nn.Linear(in_features = 1024, out_features = 10)
   
    "Forward pass"
    def forward(self,t):
        
        "input layer"
        t = t
        
        "hidden conv layer 1"
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2, stride = 2)
        
        "hidden conv layer 2"
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 2, stride = 2)
        "hidden conv layer 3"
        t = self.conv3(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size = 3, stride = 2)
        
        "hidden linear layer 1"
        t = t.reshape(-1,256*24*24)
        t = self.fc1(t)
        t = F.relu(t)
        
        "hidden linear layer 2"
        t = self.fc2(t)
        t = F.relu(t)
        
        "output layer"
        t = self.out(t)
        "t = F.softmax(t)"
        
        return t

model = Network()




In [5]:
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

        logits = model(features)
        _, predicted_labels = torch.max(logits, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


def compute_epoch_loss(model, data_loader):
    model.eval()
    curr_loss, num_examples = 0., 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(DEVICE)
            targets = targets.to(DEVICE)
            logits = model(features)
            loss = F.cross_entropy(logits, targets, reduction='sum')
            num_examples += targets.size(0)
            curr_loss += loss

        curr_loss = curr_loss / num_examples
        return curr_loss
    
    

start_time = time.time()
for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%% | Loss: %.3f' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader),
              compute_epoch_loss(model, train_loader)))


    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))


Epoch: 001/010 | Batch 0000/0391 | Cost: 2.3075
Epoch: 001/010 | Batch 0050/0391 | Cost: 1.8386
Epoch: 001/010 | Batch 0100/0391 | Cost: 1.4731
Epoch: 001/010 | Batch 0150/0391 | Cost: 1.6369
Epoch: 001/010 | Batch 0200/0391 | Cost: 1.3444
Epoch: 001/010 | Batch 0250/0391 | Cost: 1.3738
Epoch: 001/010 | Batch 0300/0391 | Cost: 1.2262
Epoch: 001/010 | Batch 0350/0391 | Cost: 1.0712
Epoch: 001/010 | Train: 60.534% | Loss: 1.115
Time elapsed: 43.52 min
Epoch: 002/010 | Batch 0000/0391 | Cost: 1.1185
Epoch: 002/010 | Batch 0050/0391 | Cost: 1.1066
Epoch: 002/010 | Batch 0100/0391 | Cost: 1.0534
Epoch: 002/010 | Batch 0150/0391 | Cost: 1.2162
Epoch: 002/010 | Batch 0200/0391 | Cost: 1.1037
Epoch: 002/010 | Batch 0250/0391 | Cost: 0.9309
Epoch: 002/010 | Batch 0300/0391 | Cost: 0.8329
Epoch: 002/010 | Batch 0350/0391 | Cost: 1.1475
Epoch: 002/010 | Train: 72.238% | Loss: 0.795
Time elapsed: 87.04 min
Epoch: 003/010 | Batch 0000/0391 | Cost: 1.0456
Epoch: 003/010 | Batch 0050/0391 | Cost: 0.7

In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

for batch_idx, (features, targets) in enumerate(test_loader):

    features = features
    targets = targets
    break

logits = model(features.to(DEVICE))
_, predicted_labels = torch.max(logits, 1)

In [ ]:
def unnormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

n_images = 10

fig, axes = plt.subplots(nrows=1, ncols=n_images, 
                         sharex=True, sharey=True, figsize=(20, 2.5))
orig_images = features[:n_images]

for i in range(n_images):
    curr_img = orig_images[i].detach().to(torch.device('cpu'))
    curr_img = unnormalize(curr_img,
                           torch.tensor([0.485, 0.456, 0.406]),
                           torch.tensor([0.229, 0.224, 0.225])) 
    curr_img = curr_img.permute((1, 2, 0))
    axes[i].imshow(curr_img)
    axes[i].set_title(classes[predicted_labels[i]])